<a href="https://colab.research.google.com/github/jahidulislam114593/Final_Ai_Project_2074_Sentiment_Analysis/blob/main/Final_Ai_Project_2074_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")

In [ ]:
df.head()

In [ ]:
import re
df['Clean'] = df.apply(lambda row: row['Text'].lower(),axis=1)

df['Clean'] = df.apply(lambda row: re.sub("@[A-Za-z0-9_]+","", row['Clean']),axis=1)
df['Clean'] = df.apply(lambda row: re.sub("#[A-Za-z0-9_]+","", row['Clean']),axis=1)

df['Clean'] = df.apply(lambda row: re.sub(r"www.\S+","", row['Clean']),axis=1)
df['Clean'] = df.apply(lambda row: re.sub(r"http\S+","", row['Clean']),axis=1)

df['Clean'] = df.apply(lambda row: re.sub("[^a-z0-9]"," ", row['Clean']),axis=1)

df = df[df['Language'] == 'en']

df[['Text','Clean','Language','Label']].head(10)

In [ ]:
df = df.sample(frac=0.40, random_state=42)
print(df.shape)

# **Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and labels (y)
X = df['Clean']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y)

# **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer with desired settings
tfidf_vectorizer = TfidfVectorizer(max_features=200, ngram_range=(1, 2))

# Fit and transform the training data
X_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data using the same vectorizer
X_test = tfidf_vectorizer.transform(X_test)

print(X_train)

# **SVC**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Create and train the Support Vector Classifier (SVC)
svc_model = SVC(kernel='linear')  # You can choose different kernels and hyperparameters
svc_model.fit(X_train, y_train)

# Make predictions on the test set and calculate accuracy
y_pred = svc_model.predict(X_test)

# Generate a classification report
svc = classification_report(y_test, y_pred)

# Print the classification report
print(svc)

              precision    recall  f1-score   support

   litigious       0.98      0.97      0.97     17918
    negative       0.94      0.94      0.94     24592
    positive       0.94      0.95      0.95     24744
 uncertainty       0.95      0.95      0.95     19877

    accuracy                           0.95     87131
   macro avg       0.95      0.95      0.95     87131
weighted avg       0.95      0.95      0.95     87131



# **Multiclass Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Make predictions on the test set
y_pred = svc_model.predict(X_test)

# Generate a confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Display the confusion matrix using a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Greys',
            xticklabels=['negative', 'positive', 'uncertainty', 'litigious'],
            yticklabels=['negative', 'positive', 'uncertainty', 'litigious'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Multiclass Confusion Matrix')
plt.show()

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create and train the Logistic Regression Classifier
lr_model = LogisticRegression(max_iter=2000)  # You can adjust hyperparameters as needed
lr_model.fit(X_train, y_train)

# Make predictions on the test set and calculate accuracy
y_pred_lr = lr_model.predict(X_test)

# Generate a classification report for Logistic Regression
lr = classification_report(y_test, y_pred_lr)

# Print the classification report for Logistic Regression
print(lr)

In [ ]:
# Make predictions on the test set
y_pred = lr_model.predict(X_test)

# Generate a confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Display the confusion matrix using a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['negative', 'positive', 'uncertainty', 'litigious'],
            yticklabels=['negative', 'positive', 'uncertainty', 'litigious'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Multiclass Confusion Matrix')
plt.show()